# PyGrid Client Worker API

### Import Libs

In [1]:
from syft.grid.client.client import connect
from syft.grid.client.grid_connection import GridHTTPConnection
from syft.core.node.domain.client import DomainClient

import syft as sy
import torch as th

sy.VERBOSE = False

### Connect performing authentication process

In [2]:
domain = connect(
    url="http://3.12.71.243:5000", # Domain Address
    credentials={"email":"admin@email.com", "password":"admin"},
    conn_type= GridHTTPConnection,) # HTTP Connection Protocol

### Create a new workers/environments

In [3]:
domain.workers.instance_type(pandas=True)

,Accelerated Computing Instances(GPU),Compute Optimized Instances,General Purpose Instances,Free Tier Instances
0,p4d.24xlarge,c5n.large,t2.2xlarge,t2.micro
1,p2.8xlarge,c5.4xlarge,t2.micro,
2,p3.2xlarge,c5n.4xlarge,t2.large,
3,p3.16xlarge,c5n.2xlarge,t2.xlarge,
4,p2.xlarge,c5a.4xlarge,t2.small,
5,p2.16xlarge,c5d.12xlarge,t2.medium,
6,p3.8xlarge,c5d.large,t2.nano,
7,,c5.xlarge,,
8,,c5.metal,,
9,,c5d.18xlarge,,


In [4]:
domain.workers.create(instance_type="t2.large")

{'msg': 'Worker created successfully!'}

### Get all Environments

In [5]:
domain.workers.all(pandas=True)

,id,state,provider,region,instance_type,address,created_at,destroyed_at
0,1,2,aws,us-east-2,t2.large,3.133.113.112:5001,2021-03-31 13:27:32.840550,2021-03-31 13:22:15.716888
1,2,2,aws,us-east-2,t2.large,3.142.210.61:5001,2021-03-31 13:39:30.726419,2021-03-31 13:22:15.716888
2,3,2,aws,us-east-2,t2.large,3.15.22.211:5001,2021-03-31 13:53:35.048078,2021-03-31 13:22:15.716888


### Get Specific Environment

In [6]:
worker = domain.workers[2]

In [7]:
print("Worker Provider: ", worker.provider)
print("Worker Instance Type: ", worker.instance_type)
print("Worker Region:", worker.region)

Worker Provider:  aws
Worker Instance Type:  t2.large
Worker Region: us-east-2


## Uploading Tensors

#### Send a tensor to the Domain

In [8]:
import torch as th

x = th.tensor([1,2,3,4,5,6])
domain_tensor_pointer = x.send(domain, pointable=True, tags=["#dataset"])

#### Check Domain Store

In [9]:
domain.store.pandas

,ID,Tags,Description,object_type
0,<UID: 19242cf1f3f44d169411a7f0aa0af1b9>,[#dataset],,<class 'torch.Tensor'>


#### Move a Domain Tensor to your temporary environment

In [11]:
domain.load(domain_tensor_pointer, worker.address)

#### Send a private tensor to your temporary environment

In [12]:
private_tensor_y = th.tensor([6,5,4,3,2,1])
worker_tensor_pointer = private_tensor_y.send(worker, pointable=True, tags=["#diff-dataset"])

#### Check Environment Store

In [13]:
worker.store.pandas

,ID,Tags,Description,object_type
0,<UID: 32ed2b67415e4b91b0fa095625109ec4>,[#dataset],,<class 'torch.Tensor'>
1,<UID: 41944b994fb342059b9a2275106b7207>,[#diff-dataset],,<class 'torch.Tensor'>


### Perform computation using private environment

In [14]:
result = worker.store["#dataset"] + worker.store["#diff-dataset"]

### Save/Send results from env to the Domain

In [15]:
worker.save(result)

In [16]:
domain.store.pandas

,ID,Tags,Description,object_type
0,<UID: 19242cf1f3f44d169411a7f0aa0af1b9>,[#dataset],,<class 'torch.Tensor'>
1,<UID: dcbc9662a7b44b0bbf95963bdc4d90b9>,"[#dataset, #diff-dataset, __add__]",,<class 'torch.Tensor'>


### Perform Request to get access to datasets

In [20]:
domain.store[1].request(reason="I would like to get this tensor")

### Evaluate data access requests

In [21]:
domain.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,"[#dataset, #diff-dataset, __add__]",I would like to get this tensor,<UID: 552995c0f7ee4bafb12df5a5c6e553db>,<UID: dcbc9662a7b44b0bbf95963bdc4d90b9>,<class 'torch.Tensor'>


In [22]:
domain.requests[0].accept()

### Retrieve the real data value

In [23]:
domain.store[1].get()

tensor([7, 7, 7, 7, 7, 7])

### Delete Environment

In [24]:
del domain.workers[1]

In [25]:
domain.workers.all(pandas=True)

,id,state,provider,region,instance_type,address,created_at,destroyed_at
0,1,3,aws,us-east-2,t2.large,3.133.113.112:5001,2021-03-31 13:27:32.840550,2021-03-31 13:56:22.009324
1,2,2,aws,us-east-2,t2.large,3.142.210.61:5001,2021-03-31 13:39:30.726419,2021-03-31 13:22:15.716888
2,3,2,aws,us-east-2,t2.large,3.15.22.211:5001,2021-03-31 13:53:35.048078,2021-03-31 13:22:15.716888
